In [1]:
# Import libraries and packages
import pandas as pd
import numpy as np
import warnings
import os
warnings.filterwarnings("ignore")

In [2]:
# Load admitting diagnoses
adm_diag = pd.read_parquet('/labs/kamaleswaranlab/ECMO/new_data/TAB5_Admitting_Diagnoses.parquet.gzip')
adm_diag.columns = ['patid', 'name', 'dob', 'mrn', 'csn', 'icd9', 'icd10', 'diagnosis', 'admission_date']
adm_diag['csn'] = adm_diag['csn'].astype(int)
adm_diag.dropna(subset=['diagnosis'], inplace=True)

In [4]:
# Add flags
adm_diag['sepsis_septicemia'] = 0
adm_diag.loc[adm_diag['diagnosis'].str.contains('sepsis|septicemia', case=False, na=False), 'sepsis_septicemia'] = 1

adm_diag['septic_shock'] = 0
adm_diag.loc[(adm_diag['diagnosis'].str.contains('septic shock', case=False, na=False)) & 
            ~(adm_diag['diagnosis'].str.contains('septicemia|without', case=False, na=False)), 'septic_shock'] = 1

adm_diag['sickle_cell'] = 0
adm_diag.loc[adm_diag['diagnosis'].str.contains('sickle', case=False, na=False), 'sickle_cell'] = 1

adm_diag['dka'] = 0
adm_diag.loc[(adm_diag['diagnosis'].str.contains('diabetes', case=False, na=False)) &
            (adm_diag['diagnosis'].str.contains('ketoacidosis', case=False, na=False)), 'dka'] = 1

adm_diag['asthmaticus'] = 0
adm_diag.loc[adm_diag['diagnosis'].str.contains('asthmaticus', case=False, na=False), 'asthmaticus'] = 1

adm_diag.to_parquet('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_models/flagged_adm_diag.parquet.gzip', compression='gzip')